In [25]:
# Import dependencies
import pandas as pd
import numpy as np
import requests
import swifter
import re

In [27]:
# Read in Billboard Top 100 dataset
df = pd.read_csv('./Resources/charts.csv')

# Convert date column to date type
df['date'] = pd.to_datetime(df['date'])
# Create new column for year
df['year'] = df['date'].dt.year
# Remove special characters from artist and song name
df['artist'] = df['artist'].str.replace(r"\(.*\)","")
# Remove double quotes from song names
df['song'] = df['song'].str.replace("\"", "")
# Remove single quotes/apostrophes from song names
df['song'] = df['song'].str.replace("'", "")

# Preview dataframe
df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,2021
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,2021
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,2021


In [28]:
# Create new dataframe of all unique Billboard charting songs
unique_billboard_tracks_df = df.groupby(['song','artist', 'year'], as_index=False).agg({'peak-rank': 'min', 
                                                                                        'weeks-on-board': 'max'})

# Create separate Billboard dataframes for each decade
billboard_1960s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1970) & (unique_billboard_tracks_df['year']>=1960)]
billboard_1970s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1980) & (unique_billboard_tracks_df['year']>=1970)]
billboard_1980s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1990) & (unique_billboard_tracks_df['year']>=1980)]
billboard_1990s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2000) & (unique_billboard_tracks_df['year']>=1990)]
billboard_2000s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2010) & (unique_billboard_tracks_df['year']>=2000)]
billboard_2010s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2020) & (unique_billboard_tracks_df['year']>=2010)]
billboard_2020s = unique_billboard_tracks_df[unique_billboard_tracks_df['year']>=2020]

# Create list of Billboard dataframes
billboard_dfs = [billboard_1960s, billboard_1970s, billboard_1980s, billboard_1990s, billboard_2000s, billboard_2010s, billboard_2020s]

In [21]:
# SPOTIFY API
# Import dependencies for Spotipy package
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Import Client ID and Client Secret from config.py
from config import cid, secret

# Initialize objects for accessing Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [22]:
# Searches spotify API by song and artist and returns Track ID
def search_spotify(song_title, artist):
    try:
        searchResults = sp.search(q=f"artist:{artist} track:{song_title}", type="track")
        track_id = searchResults['tracks']['items'][0]['id']

        return track_id
    except:
        pass

In [23]:
# Returns list of different song title combinations with/without/inside/outside parentheses
def adjust_parens(song_title):
    # Initialize empty list for song string combos
    combinations = []

    # Remove parentheses from song title
    strip_parens = ' '.join(song_title.strip(')(').split(')'))
    strip_parens = ' '.join(strip_parens.strip(' (').split('(')).replace('  ',' ')
    # Add song title 
    combinations.append(strip_parens)

    try:
        inside_parens = re.findall(r'\(.*?\)', song_title)[0].strip('()').strip()
    except IndexError:
        pass
        
    try:
        left_parens = song_title.split(f"({inside_parens})")[0].strip()
        if (left_parens not in combinations) & (len(left_parens) > 0): 
            combinations.append(left_parens)
    except:
        left_parens = "NA"

    try:
        right_parens = song_title.split(f"({inside_parens})")[1].strip()
        if (right_parens not in combinations) & (len(right_parens) > 0): 
            combinations.append(right_parens)
    except:
        right_parens = "NA"

    return combinations

In [24]:
# Create function to retrieve track ID from Spotify given the artist and song title
def get_track_id(song_title, artist):
    while True:
        # Select first artist if multiple listed with "Featuring" keyword
        if 'Featuring' in artist:
            artist = artist.split(' Featuring ')[0]
        # Select first artist if multiple listed with "with", "With", or "," substrings
        elif ' with ' in artist:
            artist = artist.split(' with ')[0]
        elif ' With ' in artist:
            artist = artist.split(' With ')[0]
        elif "," in artist:
            artist = artist.split(',')[0]

        # Make initial API search, return ID string if found
        found_id = search_spotify(song_title, artist)
        if found_id:
            return found_id

        # Search artist and song title (replacing words ending in "in" to "ing")
        song_title = re.sub(r"in\b", 'ing ', song_title)
        found_id = search_spotify(song_title, artist)
        if found_id:
            return found_id

        if '(' in song_title:
            for item in adjust_parens(song_title):
                found_id = search_spotify(item, artist)
                if found_id:
                    return found_id
                
        # Check for '/' character in song_title
        if '/' in song_title:
            # Try string on left side of '/'
            song_title = song_title.split('/')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id

            # Try string on right side of '/'
            try:
                song_title = song_title.split('/')[1]
                found_id = search_spotify(song_title, artist)
                if found_id:
                    return found_id
            except:
                pass

        # Check for '&' character in artist name
        if ' & ' in artist:
            artist = artist.split(' & ')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id
        # Check for 'X' character in artist name
        if ' X ' in artist:
            artist = artist.split(' X ')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id
        # Check for 'x' character in artist name
        if ' x ' in artist:
            artist = artist.split(' x ')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id
            
        # Print song title and artist for non-match
        if found_id:
            return found_id
        
        break

            # print(f"No ID found for '{song_title}' by {artist}")
    

In [31]:
# Returns tuple of audio features from Spotify for specified track_id
def get_audio_features(id):
    try:
        search_results = sp.audio_features(id)[0]
        danceability = search_results['danceability']
        energy = search_results['energy']
        # key = search_results['key']
        loudness = search_results['loudness']
        # mode = search_results['mode']
        speechiness = search_results['speechiness']
        acousticness = search_results['acousticness']
        instrumentalness = search_results['instrumentalness']
        liveness = search_results['liveness']
        valence = search_results['valence']
        tempo = search_results['tempo']
        duration_ms = search_results['duration_ms']
        # time_signature = search_results['time_signature']
    except:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

    return [danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, \
                valence, tempo, duration_ms]

In [29]:
sp.audio_features("4M5nCE77Qaxayuhp3fVn4V")[0]

In [9]:
# Add new "track_id" column for 1960's dataset based on Spotify API calls and export as csv
billboard_1960s['track_id'] = billboard_1960s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1960s.to_csv(f'./Resources/billboard_1960s.csv', index=False)

In [64]:
# Add new "track_id" column for 1970's dataset based on Spotify API calls and export as csv
billboard_1970s['track_id'] = billboard_1970s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1970s.to_csv(f'./Resources/billboard_1970s.csv', index=False)

Pandas Apply:   0%|          | 0/6251 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
# Add new "track_id" column for 1980's dataset based on Spotify API calls and export as csv
billboard_1980s['track_id'] = billboard_1980s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1980s.to_csv(f'./Resources/billboard_1980s.csv', index=False)

Pandas Apply:   0%|          | 0/5126 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
# Add new "track_id" column for 1990's dataset based on Spotify API calls and export as csv
billboard_1990s['track_id'] = billboard_1990s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1990s.to_csv(f'./Resources/billboard_1990s.csv', index=False)

Pandas Apply:   0%|          | 0/4405 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
# Add new "track_id" column for 2000's dataset based on Spotify API calls and export as csv
billboard_2000s['track_id'] = billboard_2000s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2000s.to_csv(f'./Resources/billboard_2000s.csv', index=False)

Pandas Apply:   0%|          | 0/4433 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
# Add new "track_id" column for 2010's dataset based on Spotify API calls and export as csv
billboard_2010s['track_id'] = billboard_2010s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2010s.to_csv(f'./Resources/billboard_2010s.csv', index=False)

Pandas Apply:   0%|          | 0/5582 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
# Add new "track_id" column for 2020's dataset based on Spotify API calls and export as csv
billboard_2020s['track_id'] = billboard_2020s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2020s.to_csv(f'./Resources/billboard_2020s.csv', index=False)

Pandas Apply:   0%|          | 0/1479 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
# Read in each decade csv file for Billboard data
df_1960s = pd.read_csv('./Resources/billboard_1960s.csv')
df_1970s = pd.read_csv('./Resources/billboard_1970s.csv')
df_1980s = pd.read_csv('./Resources/billboard_1980s.csv')
df_1990s = pd.read_csv('./Resources/billboard_1990s.csv')
df_2000s = pd.read_csv('./Resources/billboard_2000s.csv')
df_2010s = pd.read_csv('./Resources/billboard_2010s.csv')
df_2020s = pd.read_csv('./Resources/billboard_2020s.csv')

In [9]:
all_billboard_df = pd.concat([df_1960s, df_1970s, df_1980s, df_1990s, df_2000s, df_2010s, df_2020s])
all_billboard_df.to_csv('./Resources/billboard_ALL.csv', index=False)

In [10]:
# Get audio features for each dataframe and export as separate csv
def add_features_to_df(df, file_name):
    df.dropna(subset=['track_id'], inplace=True)
    df['danceability'], df['energy'], df['key'], df['loudness'], df['mode'], df['speechiness'], df['acousticness'], \
    df['instrumentalness'], df['liveness'], df['valence'], df['tempo'], df['duration_ms'], df['time_signature'] = \
                                          zip(*df['track_id'].map(get_audio_features))

    df.to_csv(f'./Resources/{file_name}_features.csv', index=False)

In [108]:
# Add feature columns to each decade dataframe and export as new csv file
add_features_to_df(df_2020s, "billboard_2020s")
add_features_to_df(df_2010s, "billboard_2010s")
add_features_to_df(df_2000s, "billboard_2000s")
add_features_to_df(df_1990s, "billboard_1990s")
add_features_to_df(df_1980s, "billboard_1980s")
add_features_to_df(df_1970s, "billboard_1970s")
add_features_to_df(df_1960s, "billboard_1960s")

In [6]:
# Read in each decade csv file for Billboard data
df_1960s = pd.read_csv('./Resources/billboard_1960s_features.csv')
df_1970s = pd.read_csv('./Resources/billboard_1970s_features.csv')
df_1980s = pd.read_csv('./Resources/billboard_1980s_features.csv')
df_1990s = pd.read_csv('./Resources/billboard_1990s_features.csv')
df_2000s = pd.read_csv('./Resources/billboard_2000s_features.csv')
df_2010s = pd.read_csv('./Resources/billboard_2010s_features.csv')
df_2020s = pd.read_csv('./Resources/billboard_2020s_features.csv')

In [7]:
# Combine all complete billboard dfs into one and export to csv file
all_billboard_features_df = pd.concat([df_1960s, df_1970s, df_1980s, df_1990s, df_2000s, df_2010s, df_2020s])
all_billboard_features_df.drop("weeks-at-no1", axis=1, inplace=True)
all_billboard_features_df.dropna(inplace=True)
all_billboard_features_df['billboard'] = 1
all_billboard_features_df.to_csv('./Resources/billboard_ALL_features.csv', index=False)

In [10]:
# Read in 1.2 Million Spotify Songs dataset
spotify_1m = pd.read_csv('./Resources/tracks_features.csv')
# all_billboard_features_df = pd.read_csv('./Resources/billboard_ALL_features.csv')
spotify_1m.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02


In [19]:
spotify_1m = spotify_1m.rename(columns={"id":"track_id", "name":"song"})
spotify_1m[['peak-rank', 'weeks-on-board', 'billboard']] = 0
# Strip brackets and single quotes from artist name
spotify_1m['artist'] = spotify_1m['artists'].str.strip("[']")

spotify_1m_filtered = spotify_1m[['song', 'artist', 'year', 'peak-rank', 'weeks-on-board', 'track_id',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature']]

spotify_1m_filtered['billboard'] = 0

spotify_1m_filtered.head()

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,song,artist,year,peak-rank,weeks-on-board,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,billboard
0,Testify,Rage Against The Machine,1999,0,0,7lmeHLHBe4nmXzuXc0HDjk,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,0
1,Guerrilla Radio,Rage Against The Machine,1999,0,0,1wsRitfRRtWyEapl0q22o8,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,0
2,Calm Like a Bomb,Rage Against The Machine,1999,0,0,1hR0fIFK2qRG3f3RF70pb7,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,0
3,Mic Check,Rage Against The Machine,1999,0,0,2lbASgTSoDO7MTuLAXlTW0,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,0
4,Sleep Now In the Fire,Rage Against The Machine,1999,0,0,1MQTmpYOZ6fcMQc56Hdo7T,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,0


In [21]:
all_songs_df = pd.concat([all_billboard_features_df, spotify_1m_filtered])

all_songs_df = all_songs_df.drop_duplicates(subset='track_id')
all_songs_df.head()

,song,artist,year,peak-rank,weeks-on-board,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,billboard
0,(1-2-3-4-5-6-7) Count The Days,Inez & Charlie Foxx,1968,76,5,4wXNtDvLZl55vlQashWQrI,0.686,0.562,5.0,-8.538,1.0,0.0504,0.589,0.000053,0.124,0.793,116.782,156886.0,4.0,1
1,(All Of A Sudden) My Heart Sings,Mel Carter,1965,38,7,6x5BiQwNlbtisITsEHa8Eu,0.419,0.546,7.0,-7.819,1.0,0.0363,0.479,0.000005,0.535,0.493,118.106,164067.0,3.0,1
2,(Baby) Hully Gully,The Olympics,1960,72,7,2CkbNJooPxUsHG4gFXKhxa,0.699,0.661,2.0,-6.090,1.0,0.0382,0.516,0.000000,0.198,0.869,115.772,218642.0,4.0,1
3,(Come round Here) Im The One You Need,The Miracles,1966,17,9,5YvokzRkNONpj5c0TIqTjQ,0.545,0.607,0.0,-9.525,1.0,0.0347,0.242,0.000000,0.779,0.767,113.880,150067.0,4.0,1
4,(Dance The) Mess Around,Chubby Checker,1961,24,7,58EirkLFB6O6wwRN9JowHB,0.511,0.694,10.0,-5.509,1.0,0.0479,0.381,0.000000,0.195,0.826,146.559,132933.0,4.0,1


In [11]:
def get_decade(row):
    if (row['year']<1970) & (row['year']>=1960):
        return "1960s"
    elif (row['year']<1980) & (row['year']>=1970):
        return "1970s"
    elif (row['year']<1990) & (row['year']>=1980):
        return "1980s"
    elif (row['year']<2000) & (row['year']>=1990):
        return "1990s"
    elif (row['year']<2010) & (row['year']>=2000):
        return "2000s"
    elif (row['year']<2020) & (row['year']>=2010):
        return "2010s"
    elif (row['year']<2020) & (row['year']>=2010):
        return "2020s"

In [24]:
# df.apply(lambda row: get_decade(row), axis=1)

all_songs_df['decade'] = all_songs_df.swifter.apply(lambda row:get_decade(row),axis=1)

Pandas Apply:   0%|          | 0/1225740 [00:00<?, ?it/s]

In [25]:
all_songs_df.head()

,song,artist,year,peak-rank,weeks-on-board,track_id,danceability,energy,key,loudness,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,billboard,decade
0,(1-2-3-4-5-6-7) Count The Days,Inez & Charlie Foxx,1968,76,5,4wXNtDvLZl55vlQashWQrI,0.686,0.562,5.0,-8.538,...,0.0504,0.589,0.000053,0.124,0.793,116.782,156886.0,4.0,1,1960s
1,(All Of A Sudden) My Heart Sings,Mel Carter,1965,38,7,6x5BiQwNlbtisITsEHa8Eu,0.419,0.546,7.0,-7.819,...,0.0363,0.479,0.000005,0.535,0.493,118.106,164067.0,3.0,1,1960s
2,(Baby) Hully Gully,The Olympics,1960,72,7,2CkbNJooPxUsHG4gFXKhxa,0.699,0.661,2.0,-6.090,...,0.0382,0.516,0.000000,0.198,0.869,115.772,218642.0,4.0,1,1960s
3,(Come round Here) Im The One You Need,The Miracles,1966,17,9,5YvokzRkNONpj5c0TIqTjQ,0.545,0.607,0.0,-9.525,...,0.0347,0.242,0.000000,0.779,0.767,113.880,150067.0,4.0,1,1960s
4,(Dance The) Mess Around,Chubby Checker,1961,24,7,58EirkLFB6O6wwRN9JowHB,0.511,0.694,10.0,-5.509,...,0.0479,0.381,0.000000,0.195,0.826,146.559,132933.0,4.0,1,1960s


In [26]:
all_songs_df.to_csv("./Resources/all_songs_final.csv", index=False)

In [4]:
all_billboard_features_df = pd.read_csv("./Resources/charts.csv")

In [8]:
import json

from sklearn.preprocessing import MinMaxScaler
features_by_year_df = all_billboard_features_df.groupby('year', as_index=False).median().drop(columns=['key', 'mode', 'time_signature', 'billboard', 'peak-rank'])
features_by_year_df


features_by_year_df

features_by_year_df = features_by_year_df.rename(columns={'weeks-on-board': 'weeks_on_board'})
features_by_year_json = features_by_year_df.to_dict('list')
features_by_year_json

with open('./templates/js/featuresByYear.js', 'w') as f:
    f.write('let featuresByYear=')
    json.dump(features_by_year_json, f)

{'year': [1960,
  1961,
  1962,
  1963,
  1964,
  1965,
  1966,
  1967,
  1968,
  1969,
  1970,
  1971,
  1972,
  1973,
  1974,
  1975,
  1976,
  1977,
  1978,
  1979,
  1980,
  1981,
  1982,
  1983,
  1984,
  1985,
  1986,
  1987,
  1988,
  1989,
  1990,
  1991,
  1992,
  1993,
  1994,
  1995,
  1996,
  1997,
  1998,
  1999,
  2000,
  2001,
  2002,
  2003,
  2004,
  2005,
  2006,
  2007,
  2008,
  2009,
  2010,
  2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2020,
  2021],
 'weeks_on_board': [9.0,
  7.0,
  8.0,
  8.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  8.0,
  8.0,
  9.0,
  9.0,
  9.0,
  8.0,
  8.0,
  10.0,
  10.0,
  10.0,
  11.0,
  12.0,
  11.0,
  11.0,
  12.0,
  12.0,
  13.0,
  13.0,
  13.0,
  13.0,
  13.0,
  13.0,
  14.0,
  14.0,
  14.0,
  14.0,
  15.0,
  15.0,
  13.0,
  14.0,
  16.0,
  18.0,
  18.0,
  17.0,
  16.0,
  14.0,
  14.0,
  14.0,
  12.0,
  10.0,
  8.0,
  7.0,
  12.0,
  12.0,
  12.5,
  11.0,
  11.0,
  8.0,
  4.0,
  5.0,
  2.0,
  4.0],
 'da

In [12]:
all_billboard_features_df['decade'] = all_billboard_features_df.swifter.apply(lambda row:get_decade(row),axis=1)
all_billboard_features_df.head()

Pandas Apply:   0%|          | 0/31874 [00:00<?, ?it/s]

,song,artist,year,peak-rank,weeks-on-board,track_id,danceability,energy,key,loudness,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,billboard,decade
0,(1-2-3-4-5-6-7) Count The Days,Inez & Charlie Foxx,1968,76,5,4wXNtDvLZl55vlQashWQrI,0.686,0.562,5.0,-8.538,...,0.0504,0.589,0.000053,0.124,0.793,116.782,156886.0,4.0,1,1960s
1,(All Of A Sudden) My Heart Sings,Mel Carter,1965,38,7,6x5BiQwNlbtisITsEHa8Eu,0.419,0.546,7.0,-7.819,...,0.0363,0.479,0.000005,0.535,0.493,118.106,164067.0,3.0,1,1960s
2,(Baby) Hully Gully,The Olympics,1960,72,7,2CkbNJooPxUsHG4gFXKhxa,0.699,0.661,2.0,-6.090,...,0.0382,0.516,0.000000,0.198,0.869,115.772,218642.0,4.0,1,1960s
3,(Come round Here) Im The One You Need,The Miracles,1966,17,9,5YvokzRkNONpj5c0TIqTjQ,0.545,0.607,0.0,-9.525,...,0.0347,0.242,0.000000,0.779,0.767,113.880,150067.0,4.0,1,1960s
4,(Dance The) Mess Around,Chubby Checker,1961,24,7,58EirkLFB6O6wwRN9JowHB,0.511,0.694,10.0,-5.509,...,0.0479,0.381,0.000000,0.195,0.826,146.559,132933.0,4.0,1,1960s


In [34]:
features = ['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms']
grouped_decade_df = all_billboard_features_df.drop(columns=['year', 'billboard'])

for feature in features:
    grouped_decade_df[feature] = MinMaxScaler().fit_transform(np.array(grouped_decade_df[feature]).reshape(-1,1))

grouped_decade_df = grouped_decade_df.groupby('decade', as_index=False).median()
grouped_decade_df = grouped_decade_df.set_index('decade')
grouped_decade_dict = grouped_decade_df.to_dict('index')

grouped_decade_df


with open('./static/js/groupedDecades.js', 'w') as f:
    f.write('let groupedDecades=')
    json.dump(grouped_decade_dict, f)


,peak-rank,weeks-on-board,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
decade,,,,,,,,,,,,,,,
1960s,43.0,7.0,0.548583,0.510617,0.454545,0.621564,1.0,0.038381,0.583919,0.000003,0.143293,0.708375,0.482758,0.103924,4.0
1970s,42.0,9.0,0.587045,0.586451,0.454545,0.614609,1.0,0.040484,0.291456,0.000083,0.115346,0.703330,0.490459,0.147402,4.0
1980s,34.0,12.0,0.641700,0.668857,0.454545,0.624843,1.0,0.039327,0.128641,0.000069,0.106707,0.708375,0.497471,0.175409,4.0
1990s,40.0,14.0,0.669028,0.658241,0.545455,0.674143,1.0,0.043113,0.087234,0.000012,0.108740,0.601413,0.472165,0.183597,4.0
2000s,41.0,15.0,0.636640,0.733064,0.454545,0.763704,1.0,0.053207,0.073666,0.000000,0.117886,0.558022,0.493666,0.164406,4.0
2010s,51.0,9.0,0.649798,0.693630,0.454545,0.755615,1.0,0.061935,0.082410,0.000000,0.116870,0.474268,0.514371,0.149974,4.0


In [29]:
all_billboard_features_df.columns

Index(['song', 'artist', 'year', 'peak-rank', 'weeks-on-board', 'track_id',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature', 'billboard', 'decade'],
      dtype='object')